In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Importing Essential Libraries
import os
import glob
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger, EarlyStopping
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
# Required Parameters
train_path = "/kaggle/input/video-classification-ucf11/testing_set/"            # Training Path
test_path = "/kaggle/input/video-classification-ucf11/testing_set/"             # Testing Path
no_of_frames = 1650                                                             # Number of Frames
ch = 4                                                                          # Model Selection Choice
epochs = 20                                                                     # Number of epochs
batch_size = 32                                                                 # Batch Size
n_classes = 11                                                                  # Number of Classes
patience = 2                                                                    # Patience for EarlyStopping
stime = int(time.time())                                                        # Defining Starting Time
categories = os.listdir(train_path)                                             # Name of each Class/Category

In [ ]:
categories.sort()
print(categories)

In [ ]:
# Defining Base Model according to choice given
# By default Model 4 [ResNet50V2] is selected
if ch == 1:
    from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
    base_model = ResNet50(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
elif ch == 2:
    from tensorflow.keras.applications.resnet import ResNet101, preprocess_input
    base_model = ResNet101(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
elif ch == 3:
    from tensorflow.keras.applications.resnet import ResNet152, preprocess_input
    base_model = ResNet150(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
elif ch == 4:
    from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
    base_model = ResNet50V2(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
elif ch == 5:
    from tensorflow.keras.applications.resnet_v2 import ResNet101V2, preprocess_input
    base_model = ResNet101V2(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
elif ch == 6:
    from tensorflow.keras.applications.resnet_v2 import ResNet152V2, preprocess_input
    base_model = ResNet150V2(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
elif ch == 7:
    from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
    base_model = MobileNet(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
elif ch == 8:
    from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
    base_model = MobileNetV2(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.5)(x)
# x = Dense(512, activation = 'relu')(x)
# x = Dense(256, activation = 'relu')(x)
preds = Dense(n_classes, activation = 'softmax')(x)

In [ ]:
model = Model(inputs = base_model.input, outputs = preds)

In [ ]:
# Printing the names of each layer
for i, layer in enumerate(model.layers):
    print(i, layer.name)

In [ ]:
# Model Summary
print(model.summary())

In [ ]:
# Setting each layer as trainable
for layer in model.layers:
    layer.trainable = True

In [ ]:
# Setting 1/3 layers as trainable
# for layer in model.layers[:65]:
#     layer.trainable = False
# for layer in model.layers[65:]:
#     layer.trainable = True

In [ ]:
# Defining Image Data Generators
train_datagenerator = ImageDataGenerator(preprocessing_function = preprocess_input,
                                         validation_split = 0.2)

test_datagenerator = ImageDataGenerator(preprocessing_function = preprocess_input)

train_generator = train_datagenerator.flow_from_directory(train_path,
                                                          target_size = (224, 224),
                                                          color_mode = 'rgb',
                                                          batch_size = batch_size,
                                                          class_mode = 'categorical',
                                                          shuffle = True)

validation_generator = train_datagenerator.flow_from_directory(train_path,
                                                               target_size = (224, 224),
                                                               color_mode = 'rgb',
                                                               batch_size = batch_size,
                                                               class_mode = 'categorical',
                                                               subset = 'validation')

test_generator = test_datagenerator.flow_from_directory(test_path,
                                                        target_size = (224, 224),
                                                        color_mode = 'rgb',
                                                        class_mode = 'categorical')

In [ ]:
print(train_generator.class_indices)
print(validation_generator.class_indices)
print(test_generator.class_indices)

In [ ]:
# Compiling the Model
model.compile(optimizer = "Adam",
              loss = "categorical_crossentropy",
              metrics = ["accuracy"])

In [ ]:
# Creating a timestamp directory
try:
    os.mkdir("{}_{}b_{}e".format(stime, batch_size, epochs))
except:
    print("Directory already present...")

In [ ]:
# CSVLogger
filename = "{}_{}b_{}\\file.csv".format(stime, batch_size, epochs)
csv_log = CSVLogger(filename)

In [ ]:
# Early Stopping
early_stopping = EarlyStopping(patience = patience)

In [ ]:
# Tensorboard
tensorboard = TensorBoard(log_dir = "{}_{}b_{}e\logs".format(stime, batch_size, epochs))

In [ ]:
# Defining Model Checkpoint
checkpoint_name = "{}_{}b_{}e".format(stime, batch_size, epochs)
checkpoint_path = checkpoint_name + "\cp-{epoch:04d}-{accuracy:.4f}a-{loss:.4f}l-{val_accuracy:.4f}va-{val_loss:.4f}vl.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
model_checkpoint = ModelCheckpoint(checkpoint_path)

In [ ]:
# Training the Model
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = epochs,
                    callbacks = [model_checkpoint, tensorboard, csv_log, early_stopping])

In [ ]:
# Plotting the Graph
model_history = pd.DataFrame(history.history)
model_history.plot()

In [ ]:
# # Loading Model
# from tensorflow.keras.models import load_model
# model = load_model(r"foldername/filename.h5") # Enter your model here

In [ ]:
# Evaluating Model's Performance
history2 = model.evaluate(test_generator)

# Testing on testing_set/

In [ ]:
# # Loading Tensorflow.Keras Model
# model = tf.keras.models.load_model("foldername/filename.h5") # Enter your model here

In [ ]:
# Image Data Generator
test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator = test_datagen.flow_from_directory(test_path,
                                                  target_size = (224, 224),
                                                  color_mode = "rgb",
                                                  shuffle = False,
                                                  class_mode = 'categorical',
                                                  batch_size = 1)

In [ ]:
activities = test_generator.class_indices
print(activities)

In [ ]:
def get_activity(val):
    for key, value in activities.items():
        if val == value:
            return key
    return "Invalid"

In [ ]:
filenames = test_generator.filenames
nb_samples = len(filenames)

In [ ]:
predict = model.predict(test_generator, steps = nb_samples, verbose = 1)

In [ ]:
y_pred = []
for val in predict:
    y_pred.append(get_activity(np.argmax(val)))

y_true = []
for file in filenames:
    y_true.append(file.split("\\")[0])

In [ ]:
cm = confusion_matrix(y_true, y_pred)

print(precision_score(y_true, y_pred, average = 'macro'))
print(recall_score(y_true, y_pred, average = 'macro'))
print(f1_score(y_true, y_pred, average = 'macro'))

print(precision_score(y_true, y_pred, average = 'micro'))
print(recall_score(y_true, y_pred, average = 'micro'))
print(f1_score(y_true, y_pred, average = 'micro'))

In [ ]:
# Making a Classification Report
print(classification_report(y_true, y_pred))

dataframe = pd.DataFrame(cm)
inv_dict = {v: k for k, v in activities.items()} 
dataframe = dataframe.rename(index = inv_dict)
dataframe = dataframe.rename(columns = inv_dict)

In [ ]:
# Saving Confusion Matrix in CSV format
dataframe.to_csv("Perfomance Confusion Matrix.csv")